In [ ]:
import os
import findspark
findspark.init()
from dotenv import load_dotenv
load_dotenv()


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pyspark.pandas as ps
import pandas as pd
#import mlflow

from warnings import filterwarnings
filterwarnings("ignore")

In [ ]:
spark = SparkSession.builder.appName("Analytic Contracts App").getOrCreate()

In [ ]:
spark

In [ ]:
#mlflow.set_experiment("test-experiment")
#mlflow.autolog()

In [ ]:
# read data
df_spark = spark.read.options(header='True', inferSchema='True').csv(os.getenv("dataPath"))

In [ ]:
df_spark.head(5)

In [ ]:
df_spark.tail(5)

In [ ]:
df_spark.describe().show()

In [ ]:
df_spark.printSchema()

In [ ]:
df_spark.dtypes

In [ ]:
# total columns
print("Total columns are:", len(df_spark.columns), "columns")

In [ ]:
# select all null values
df_spark.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_spark.columns]).show()

In [ ]:

print("unique values:", len(df_spark.select("Nombre del Procedimiento").distinct().collect()))

In [ ]:
df_spark.select("Nombre del Procedimiento").distinct().show()

In [ ]:
for col in df_spark.columns:
    mode_colum = df_spark.groupby(col).count().orderBy("count", ascending=False).first()[0]
    print(f"The mode for column {col} is: {mode_colum}")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
base_price = df_spark.select(df_spark["Precio Base"].cast(IntegerType()).alias('base_price'))
base_price.show(5)

In [ ]:
figure = plt.figure(figsize=(10, 6))
plt.hist(base_price.collect(), bins=20, edgecolor="black")
plt.title("Distribution of Base Prices")
plt.xlabel("Base Price")
plt.ylabel("Frequency")
plt.grid(True)
plt.show()

In [ ]:
# plot distribution for entity
import numpy as np
figure2 = plt.figure(figsize=(8, 6))
plt.title('Entity Distribution')
plt.xlabel('Entity')
plt.ylabel('Frequency')
plt.bar(df_spark.select("Entidad").collect(), bins=20, edgecolor='black')
display(figure2)

In [ ]:
ps_df = df_spark.pandas_api()

In [ ]:
ps_df.head(5)

In [ ]:
ps_df.tail()

In [ ]:
ps_df["Precio Base"].plot.line()

In [ ]:
ps_df["Duracion"].astype("int16").plot.bar()

In [ ]:
ps_df.groupby(["Ciudad Entidad"])["Proveedores Invitados"].mean()


In [ ]:
ps_df.groupby(["Ciudad Entidad"])["Estado del Procedimiento"].count().sort_values(ascending=False)